## Built-In Evaluators - Measuring Agent Performance with Strands Evals

This tutorial introduces the complete toolkit of built-in evaluators provided by Strands Evals. You'll learn how to measure different aspects of agent performance using standardized evaluation metrics, from response quality to tool selection accuracy.

### What You'll Learn
- Understand the purpose of each built-in evaluator
- Apply OutputEvaluator with domain-specific rubrics
- Use trace-based evaluators (HelpfulnessEvaluator, GoalSuccessRateEvaluator, ToolSelectionAccuracyEvaluator, ToolParameterAccuracyEvaluator)
- Analyze agent reasoning with TrajectoryEvaluator
- Compare evaluation results across different metrics

### Tutorial Details

| Information         | Details                                                                       |
|:--------------------|:------------------------------------------------------------------------------|
| Tutorial type       | Beginner - Introduction to built-in evaluation metrics                        |
| Tutorial components | Recipe Bot agent, 6 built-in evaluators, results visualization               |
| Tutorial vertical   | Agent Evaluation                                                              |
| Example complexity  | Easy                                                                          |
| SDK used            | Strands Agents, Strands Evals                                                 |

### Understanding Built-In Evaluators

Evaluating agent performance is complex because agents operate across multiple dimensions—they generate responses, complete tasks, and use tools to achieve goals. A single metric can't capture all aspects of agent behavior, which is why Strands Evals provides six specialized built-in evaluators.

| Evaluator | Type | Use When | Measures | Requirements |
|:----------|:-----|:---------|:---------|:-------------|
| **OutputEvaluator** | Output-based | Verify correct, complete answers | Correctness, completeness, relevance via custom rubrics | None |
| **HelpfulnessEvaluator** | Trace-based | Ensure agent is genuinely useful | Practical value, clarity, actionability (7-point scale) | OpenTelemetry and Session mapping |
| **GoalSuccessRateEvaluator** | Trace-based | Track task completion rates | Binary success/failure against defined goals | OpenTelemetry and Session mapping |
| **ToolSelectionAccuracyEvaluator** | Trace-based | Verify proper tool selection | Whether agent chose the right tools | OpenTelemetry and Session mapping |
| **ToolParameterAccuracyEvaluator** | Trace-based | Validate tool parameter usage | Correctness of tool parameter values | OpenTelemetry and Session mapping |
| **TrajectoryEvaluator** | Trajectory-based | Understand agent reasoning | Quality of reasoning steps and action sequences | Trajectory extractor |

#### Important API Note

**ONE Evaluator Per Dataset**: Each Dataset accepts exactly ONE evaluator object. To demonstrate multiple evaluators, we run separate evaluation rounds.

### Environment Setup

Configure AWS region and model settings for this tutorial.

In [ ]:
import boto3

# AWS Configuration
session = boto3.Session()
AWS_REGION = session.region_name or 'us-east-1'
DEFAULT_MODEL = 'us.anthropic.claude-3-7-sonnet-20250219-v1:0'

### Setup and Imports

Import all necessary libraries for agent creation and evaluation.

In [ ]:
# Standard imports
import json
from typing import List, Dict

# Strands imports
from strands import Agent, tool

# Strands Evals imports
from strands_evals import Dataset, Case
from strands_evals.evaluators import (
    OutputEvaluator,
    HelpfulnessEvaluator,
    GoalSuccessRateEvaluator,
    ToolSelectionAccuracyEvaluator,
    ToolParameterAccuracyEvaluator,
    TrajectoryEvaluator
)
from strands_evals.extractors import tools_use_extractor

# Display utilities
from IPython.display import Markdown, display
import pandas as pd

### Recipe Bot Agent

We'll use a Recipe Bot agent to demonstrate built-in evaluators. This agent helps users find recipes and answers cooking questions using web search.

In [ ]:
from ddgs import DDGS
from ddgs.exceptions import DDGSException, RatelimitException
import time

# Define a websearch tool
@tool
def websearch(
    keywords: str, region: str = "us-en", max_results: int | None = None
) -> str:
    """Search the web to get updated information.
    Args:
        keywords (str): The search query keywords.
        region (str): The search region: wt-wt, us-en, uk-en, ru-ru, etc..
        max_results (int | None): The maximum number of results to return.
    Returns:
        List of dictionaries with search results.
    """
    try:
        time.sleep(15)
        results = DDGS().text(keywords, region=region, max_results=max_results)
        return results if results else "No results found."
    except RatelimitException:
        return "RatelimitException: Please try again after a short delay."
    except DDGSException as d:
        return f"DuckDuckGoSearchException: {d}"
    except Exception as e:
        return f"Exception: {e}"


# System prompt for Recipe Bot
RECIPE_BOT_SYSTEM_PROMPT = """You are RecipeBot, a helpful cooking assistant.
Help users find recipes based on ingredients and answer cooking questions.
Use the websearch tool to find recipes when users mention ingredients or to look up cooking information."""

### Test the Agent

Before evaluating, let's verify the agent works correctly with a simple test query.

In [ ]:
# Create a test agent instance
test_agent = Agent(
    system_prompt=RECIPE_BOT_SYSTEM_PROMPT,
    tools=[websearch],
    model=DEFAULT_MODEL
)

# Test with a simple query
test_query = "What can I make with chicken and tomatoes?"
test_response = test_agent(test_query)

### Create Test Cases

We'll create test cases with domain-specific expectations for Recipe Bot evaluation.

In [ ]:
# Create test cases for evaluation
test_cases = [
    Case(
        name="Recipe Search - Simple Ingredients",
        input="I have chicken and broccoli. What can I cook?",
        expected_output="A helpful response with recipe suggestions that include chicken and broccoli, with cooking instructions or search results.",
        metadata={
            "goal": "Find recipes using specified ingredients",
            "expected_tools": ["websearch"],
            "expected_tool_params": {
                "websearch": {
                    "keywords": ["chicken", "broccoli", "recipe"]
                }
            }
        }
    ),
    Case(
        name="Cooking Technique Question",
        input="How do I properly sear a steak?",
        expected_output="Clear instructions on steak searing technique, including temperature, timing, and tips for achieving a good sear.",
        metadata={
            "goal": "Learn proper steak searing technique",
            "expected_tools": ["websearch"],
            "expected_tool_params": {
                "websearch": {
                    "keywords": ["sear", "steak"]
                }
            }
        }
    ),
    Case(
        name="Dietary Restriction Recipe",
        input="Can you find me a vegan pasta recipe?",
        expected_output="One or more vegan pasta recipes with ingredients and preparation steps.",
        metadata={
            "goal": "Find vegan pasta recipes",
            "expected_tools": ["websearch"],
            "expected_tool_params": {
                "websearch": {
                    "keywords": ["vegan", "pasta", "recipe"]
                }
            }
        }
    )
]

### OpenTelemetry Setup

Trace-based evaluators (HelpfulnessEvaluator, GoalSuccessRateEvaluator, ToolSelectionAccuracyEvaluator, ToolParameterAccuracyEvaluator) require OpenTelemetry setup to capture agent execution traces.

In [ ]:
from strands_evals.telemetry import StrandsEvalsTelemetry
from strands_evals.mappers import StrandsInMemorySessionMapper

# Setup telemetry - CORRECT WAY per README
telemetry = StrandsEvalsTelemetry().setup_in_memory_exporter()

### Evaluation Round 1: OutputEvaluator

OutputEvaluator assesses response quality using domain-specific rubrics. For Recipe Bot, we check for ingredient lists, cooking instructions, and timing information.

In [ ]:
# Create OutputEvaluator with domain-specific rubric
output_evaluator = OutputEvaluator(
    rubric="""Recipe responses should include:
    1. Clear ingredient list with quantities (0-0.3 points)
    2. Step-by-step cooking instructions (0-0.4 points)
    3. Cooking time/temperature if applicable (0-0.3 points)
    Score proportionally based on completeness."""
)

# Create dataset with OutputEvaluator
output_dataset = Dataset[str, str](cases=test_cases, evaluator=output_evaluator)

# Simple task function (no OTEL needed for OutputEvaluator)
def simple_task(case: Case) -> str:
    agent = Agent(
        system_prompt=RECIPE_BOT_SYSTEM_PROMPT,
        tools=[websearch],
        model=DEFAULT_MODEL
    )
    return str(agent(case.input))

# Run evaluation
output_report = output_dataset.run_evaluations(simple_task)

In [ ]:
# Display OutputEvaluator results
output_report.run_display()

### Evaluation Round 2: HelpfulnessEvaluator

HelpfulnessEvaluator measures how useful the agent's response is to users on a 7-point scale. This evaluator requires OpenTelemetry Session data.

In [ ]:
# Create HelpfulnessEvaluator
helpfulness_evaluator = HelpfulnessEvaluator()

# Create dataset
helpfulness_dataset = Dataset[str, str](cases=test_cases, evaluator=helpfulness_evaluator)

# Task function with OTEL support
import uuid

def trace_task(case: Case) -> dict:
    telemetry.in_memory_exporter.clear()
    session_id = str(uuid.uuid4())  # Generate unique session ID
    agent = Agent(
        system_prompt=RECIPE_BOT_SYSTEM_PROMPT,
        tools=[websearch],
        model=DEFAULT_MODEL,
        trace_attributes={"session.id": session_id},
        callback_handler=None
    )
    response = agent(case.input)
    
    # Force flush all spans to ensure they're captured
    telemetry.tracer_provider.force_flush()

    # Map spans to Session
    finished_spans = telemetry.in_memory_exporter.get_finished_spans()
    mapper = StrandsInMemorySessionMapper()
    session = mapper.map_to_session(finished_spans, session_id=session_id)

    return {"output": str(response), "trajectory": session}

# Run evaluation
helpfulness_report = helpfulness_dataset.run_evaluations(trace_task)

In [ ]:
helpfulness_report.run_display()

### Evaluation Round 3: GoalSuccessRateEvaluator

GoalSuccessRateEvaluator determines if the agent successfully completed the user's stated goal (binary success/failure).

In [ ]:
# Create GoalSuccessRateEvaluator
goal_evaluator = GoalSuccessRateEvaluator()

# Create dataset
goal_dataset = Dataset[str, str](cases=test_cases, evaluator=goal_evaluator)

# Run evaluation (reuse trace_task function)
goal_report = goal_dataset.run_evaluations(trace_task)


In [ ]:
# Display GoalSuccessRateEvaluator results
goal_report.run_display()

### Evaluation Round 4: ToolSelectionAccuracyEvaluator

ToolSelectionAccuracyEvaluator validates that the agent selected the correct tools for the task.

In [ ]:
# Create ToolSelectionAccuracyEvaluator
tool_selection_evaluator = ToolSelectionAccuracyEvaluator()

# Create dataset
tool_selection_dataset = Dataset[str, str](cases=test_cases, evaluator=tool_selection_evaluator)

# Run evaluation
tool_selection_report = tool_selection_dataset.run_evaluations(trace_task)


In [ ]:
# Display ToolSelectionAccuracyEvaluator results
tool_selection_report.run_display()

### Evaluation Round 5: ToolParameterAccuracyEvaluator

ToolParameterAccuracyEvaluator checks if the agent used tool parameters correctly (e.g., proper search keywords).

In [ ]:
# Create ToolParameterAccuracyEvaluator
tool_parameter_evaluator = ToolParameterAccuracyEvaluator()

# Create dataset
tool_parameter_dataset = Dataset[str, str](cases=test_cases, evaluator=tool_parameter_evaluator)

# Run evaluation
tool_parameter_report = tool_parameter_dataset.run_evaluations(trace_task)


In [ ]:
# Display ToolParameterAccuracyEvaluator results
tool_parameter_report.run_display()

### Evaluation Round 6: TrajectoryEvaluator

TrajectoryEvaluator analyzes the sequence of actions (tool calls) the agent took to reach its conclusion.

In [ ]:
# Create TrajectoryEvaluator with domain-specific rubric
trajectory_evaluator = TrajectoryEvaluator(
    rubric="""Agent should:
    1. Understand user's ingredients/dietary needs
    2. Search web with relevant recipe keywords
    3. Synthesize results into actionable recommendations
    Score: 1.0 if all steps present and logical, 0.5 if partially correct, 0.0 if flawed."""
)

# Create dataset (use single test case for demonstration)
trajectory_dataset = Dataset[str, str](cases=[test_cases[0]], evaluator=trajectory_evaluator)

# Task function with trajectory extraction
def trajectory_task(case: Case) -> dict:
    agent = Agent(
        system_prompt=RECIPE_BOT_SYSTEM_PROMPT,
        tools=[websearch],
        model=DEFAULT_MODEL
    )
    response = agent(case.input)

    # Update trajectory description
    trajectory_evaluator.update_trajectory_description(
        tools_use_extractor.extract_tools_description(agent)
    )

    # Extract trajectory from agent messages
    trajectory = tools_use_extractor.extract_agent_tools_used_from_messages(agent.messages)
    return {"output": str(response), "trajectory": trajectory}

# Run evaluation
trajectory_report = trajectory_dataset.run_evaluations(trajectory_task)


In [ ]:
# Display TrajectoryEvaluator results
trajectory_report.run_display()

### Summary: Comparing All Evaluators

Let's create a summary table comparing results from all six evaluators.

In [ ]:
# Create summary comparison table
summary_data = {
    "Evaluator": [
        "OutputEvaluator",
        "HelpfulnessEvaluator",
        "GoalSuccessRateEvaluator",
        "ToolSelectionAccuracyEvaluator",
        "ToolParameterAccuracyEvaluator",
        "TrajectoryEvaluator"
    ],
    "Overall Score": [
        f"{output_report.overall_score:.2f}",
        f"{helpfulness_report.overall_score:.2f}",
        f"{goal_report.overall_score:.2f}",
        f"{tool_selection_report.overall_score:.2f}",
        f"{tool_parameter_report.overall_score:.2f}",
        f"{trajectory_report.overall_score:.2f}"
    ],
    "Type": [
        "Output-based",
        "Trace-based",
        "Trace-based",
        "Trace-based",
        "Trace-based",
        "Trajectory-based"
    ],
    "What It Measures": [
        "Response quality (ingredients, instructions, timing)",
        "User satisfaction (7-point scale)",
        "Goal completion (binary success/failure)",
        "Correct tool selection",
        "Correct tool parameters (keywords)",
        "Action sequence quality"
    ],
    "Requirements": [
        "None",
        "OpenTelemetry + Session",
        "OpenTelemetry + Session",
        "OpenTelemetry + Session",
        "OpenTelemetry + Session",
        "Trajectory extractor"
    ]
}

summary_df = pd.DataFrame(summary_data)
display(Markdown("### Built-In Evaluator Comparison"))
display(summary_df)


#### Production Recommendations

For comprehensive agent evaluation:
1. Start with OutputEvaluator using domain-specific rubrics
2. Add HelpfulnessEvaluator and GoalSuccessRateEvaluator for user-centric metrics
3. Use tool evaluators if your agent has multiple tools or complex tool usage
4. Apply TrajectoryEvaluator for debugging and reasoning analysis

### Summary

You've successfully learned how to use built-in evaluators provided by Strands Evals.

In the next tutorial, you'll learn how to create custom evaluators for specialized evaluation needs beyond what the built-in evaluators provide.